In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
random_seed = 42

In [18]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [19]:
X_train

,game_time,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,...,d5_hero_id_107.0,d5_hero_id_108.0,d5_hero_id_109.0,d5_hero_id_110.0,d5_hero_id_111.0,d5_hero_id_112.0,d5_hero_id_113.0,d5_hero_id_114.0,d5_hero_id_119.0,d5_hero_id_120.0
7676,675.0,0.0,1.0,0.0,10.0,3357.0,48.0,4740.0,0.0,920.0,...,False,False,False,False,False,False,False,False,False,False
1257,203.0,0.0,0.0,0.0,4.0,1218.0,19.0,1130.0,604.0,980.0,...,False,False,False,False,False,False,False,False,False,False
24896,1129.0,2.0,7.0,6.0,5.0,4190.0,25.0,4582.0,840.0,840.0,...,False,False,False,False,False,False,False,False,False,False
7056,1768.0,1.0,4.0,6.0,11.0,11583.0,147.0,15354.0,1950.0,1950.0,...,False,False,False,False,False,False,False,False,False,False
21579,1489.0,2.0,5.0,2.0,4.0,8585.0,79.0,9409.0,1200.0,1200.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23444,2434.0,2.0,10.0,20.0,0.0,9649.0,37.0,14621.0,1420.0,1420.0,...,False,False,False,False,False,False,False,False,False,False
9392,1040.0,8.0,4.0,7.0,6.0,7110.0,57.0,8613.0,1440.0,1440.0,...,False,False,False,False,False,False,False,False,False,False
1499,2102.0,4.0,12.0,3.0,2.0,9826.0,102.0,12261.0,798.0,1300.0,...,False,False,False,False,False,False,False,False,False,False
27569,1148.0,1.0,7.0,7.0,0.0,3577.0,4.0,3350.0,780.0,780.0,...,False,False,False,False,False,False,False,False,False,False


In [20]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
#parameters = {
#    'criterion' : ['entropy'],
#}
model = BaggingClassifier()
#model = GridSearchCV(cf, param_grid=parameters, cv= 5)
model.fit(X_train, Y_train)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier()

In [21]:
#model.best_params_

In [22]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict(X_test))}")

Score : 0.6558666247247562
ROC-AUC score : 0.6576076049873918


In [23]:
model.predict(X_test)

array([0, 0, 1, ..., 1, 1, 0])

In [24]:
ut.write(model)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- d1_hero_id_1
- d1_hero_id_10
- d1_hero_id_100
- d1_hero_id_101
- d1_hero_id_102
- ...
Feature names seen at fit time, yet now missing:
- d1_hero_id_1.0
- d1_hero_id_10.0
- d1_hero_id_100.0
- d1_hero_id_101.0
- d1_hero_id_102.0
- ...
